In [3]:
code = """
# app.py
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import streamlit as st
import pandas as pd
import sqlite3
import speech_recognition as sr
import pyttsx3
import os
from dotenv import load_dotenv
import requests
from googleapiclient.discovery import build

# Initialize FastAPI
app = FastAPI()

# Load the pretrained model and tokenizer
model_name = "microsoft/CodeGPT-small-java"  # Example model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

class CodeRequest(BaseModel):
    prompt: str
    language: str
    max_length: int = 100  # Default max length for generated code

class CodeResponse(BaseModel):
    code: str

@app.post("/generate_code", response_model=CodeResponse)
async def generate_code(request: CodeRequest):
    try:
        # Modify the prompt based on the requested language
        full_prompt = f"Generate a code snippet in {request.language}: {request.prompt}"
        
        # Encode the input prompt
        inputs = tokenizer.encode(full_prompt, return_tensors="pt")

        # Generate code
        outputs = model.generate(inputs, max_length=request.max_length, num_return_sequences=1)
        
        # Decode the generated code
        generated_code = tokenizer.decode(outputs[0], skip_special_tokens=True)

        return CodeResponse(code=generated_code)
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

@app.get("/")
async def root():
    return {"message": "Welcome to the Multi-Language Code Generator API!"}


# Streamlit Code
load_dotenv()
youtube_api_key = os.getenv("YOUTUBE_API_KEY")  # Ensure this is set in your .env file

# SQLite DB setup
conn = sqlite3.connect('chat_history.db')
c = conn.cursor()
c.execute('''CREATE TABLE IF NOT EXISTS conversations (role TEXT, content TEXT)''')
conn.commit()

# Text-to-Speech
engine = pyttsx3.init()

def save_conversation(role, content):
    c.execute('INSERT INTO conversations (role, content) VALUES (?, ?)', (role, content))
    conn.commit()

def fetch_conversation_history():
    c.execute('SELECT role, content FROM conversations ORDER BY ROWID DESC LIMIT 10')
    return [{"role": r, "content": c} for r, c in reversed(c.fetchall())]

def search_youtube(query):
    youtube = build('youtube', 'v3', developerKey=youtube_api_key)
    request = youtube.search().list(q=query, part='snippet', type='video', maxResults=3)
    response = request.execute()
    return response['items']

# Streamlit UI
st.set_page_config(page_title="DS/ML Chatbot", layout="centered")
st.title("🤖 AI-Powered Data Science & ML Assistant")
st.write("Welcome to the Multi-Language Code Generator API!")

# User input for code generation
user_input = st.text_input("💬 Ask me to generate code (any language):")
language = st.selectbox("Select programming language:", ["Python", "Java", "JavaScript", "C++", "Ruby"])

if user_input and language:
    with st.spinner("Generating code..."):
        # Send request to FastAPI server
        response = requests.post(
            "http://127.0.0.1:8000/generate_code",  # Adjust the URL if needed
            json={"prompt": user_input, "language": language, "max_length": 100}
        )
        
        if response.status_code == 200:
            code_output = response.json().get("code")
            st.success("✅ Here's your generated code:")
            st.code(code_output, language=language.lower())
        else:
            st.error("Error generating code.")

"""

In [5]:
with open("app.py", "w", encoding="utf-8") as f:
    f.write(code)

print("File saved as app.py ✅")

File saved as app.py ✅


In [7]:
# how can run this code in tour terminal 
# you can change the llms pretrained model with your need and also make your own model
#in terminal do 
# import pyaudio for speak to search
# cd " name where your file saved"
# dir to cheack file
# streamlit run your file saving name.py
# import any thing according to you in your terminal window to run this AI code